In [ ]:
# courant criteria
import numpy as np
1/np.sqrt(3*(1/6e-3)**2)/299792458

In [1]:
import pyfdtd3d.structuredpoints_cupy as fdtd
import pyfdtd3d.structuredpoints_ui as ui
from pyfdtd3d.base import sin3, vsrc
import time

class MySolver(ui.Solver):
    def __init__(self, dt, dimensions, spacing, pos, freq):
        self.freq = freq # [Hz]
        self.timeSeries = t = fdtd.TimeSeries(dt)
        self.grid = g = fdtd.Grid(
            dimensions=(100, 100, 51), # [cells]
            spacing=(6e-3, 6e-3, 6e-3), # [mm]
        )
        self.yee = y = fdtd.Yee(grid=g)
        self.material = m = fdtd.Material(*g.dimensions)
        self.hcalc = fdtd.HCalc(yee=y, time=t)
        self.ecalc = fdtd.ECalc(yee=y, time=t, material=m)
        self.currentz = fdtd.CurrentZ(yee=y, pos=pos)
    def calculate(self):
        y, t, hc, ec, cz = self.yee, self.timeSeries, self.hcalc, self.ecalc, self.currentz
        dx, dy, dz = y.grid.spacing
        t.progress()
        hc.calculate()
        t.progress()
        ec.calculate()
        y.ez[50, 50, 26:37] = 0.
        y.ez[50, 50, 14:25] = 0.
        y.ez[50, 50, 25] = vsrc(voltage=sin3(t.current_time, self.freq), current=cz.get(), delta=dz)
    def reset(self):
        g = self.grid
        pos = self.currentz._pos
        dt = self.timeSeries.dt
        self.timeSeries = t = fdtd.TimeSeries(dt)
        self.yee = y = fdtd.Yee(grid=g)
        self.material = m = fdtd.Material(*g.dimensions)
        self.hcalc = fdtd.HCalc(yee=y, time=t) # y を変更可能に
        self.ecalc = fdtd.ECalc(yee=y, time=t, material=m) # y を変更可能に
        self.currentz = fdtd.CurrentZ(yee=y, pos=pos) # y, pos を変更可能に

s = MySolver(
    dt=0.01e-9, # s
    dimensions=(100, 100, 51), # [cells]
    spacing=(6e-3, 6e-3, 6e-3), # [mm]
    pos=(50, 50, 25),
    freq=1e+9,
)

p = ui.Plotter(
    yee=s.yee,
    slices=(50, slice(None), slice(None)),
    mapper=ui.LinearColorMapper(palette='Viridis256', low=-1, high=1),
)

u = ui.ControlPanel(s, p)

Loading BokehJS ...